In [12]:
import os
from google.cloud import bigquery
from google.oauth2 import service_account

import pandas as pd
import matplotlib.pyplot as plt

from sqlalchemy import create_engine
from sqlalchemy.types import Integer, String, Float, DateTime
import seaborn as sns
from prophet import Prophet

# Configurações de visualização
%matplotlib inline
sns.set(style="whitegrid")

In [13]:
service_account_path = 'tfm-sa.json'
credentials = service_account.Credentials.from_service_account_file(service_account_path)
project_id = 'perseverance-332400'
dataset_id = 'TFM'
table_id = 'ds_market'
full_table_id = f'{project_id}.{dataset_id}.{table_id}'
client = bigquery.Client(project=project_id, credentials=credentials)

In [14]:
sample_query = f"""
SELECT *
FROM `{full_table_id}`
WHERE EXTRACT(YEAR FROM date) = 2015;
"""

def executar_consulta(sql, client):
    query_job = client.query(sql)
    return query_job.to_dataframe()

df = executar_consulta(sample_query, client)

In [15]:
print(df.head())

                        id               item   category_x     department  \
0   ACCESORIES_1_048_BOS_1   ACCESORIES_1_048   ACCESORIES   ACCESORIES_1   
1   ACCESORIES_1_246_BOS_1   ACCESORIES_1_246   ACCESORIES   ACCESORIES_1   
2   ACCESORIES_1_285_BOS_1   ACCESORIES_1_285   ACCESORIES   ACCESORIES_1   
3   ACCESORIES_2_096_BOS_1   ACCESORIES_2_096   ACCESORIES   ACCESORIES_2   
4  SUPERMARKET_2_264_BOS_1  SUPERMARKET_2_264  SUPERMARKET  SUPERMARKET_2   

       store store_code  region       d  sales yearweek  \
0  South_End      BOS_1  Boston  d_1528      4   201514   
1  South_End      BOS_1  Boston  d_1467      0   201506   
2  South_End      BOS_1  Boston  d_1607      0   201526   
3  South_End      BOS_1  Boston  d_1743      0   201545   
4  South_End      BOS_1  Boston  d_1538      1   201516   

                       date   event   category_y  sell_price  
0 2015-04-05 00:00:00+00:00  Easter   ACCESORIES      0.6118  
1 2015-02-03 00:00:00+00:00    None   ACCESORIES     14.

In [4]:
df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)

In [5]:
product_id = 'ACCESORIES_1_238'
store_id = 'Harlem'
filtered_df = df[(df['item'] == product_id) & (df['store'] == store_id)]

In [ ]:
filtered_df['date'] = pd.to_datetime(filtered_df['date']).dt.tz_localize(None)
filtered_df.set_index('date', inplace=True)
agg_data = filtered_df.resample('W').agg({'sales': 'sum'}).reset_index()
agg_data.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)

# Tratar valores ausentes
agg_data['y'] = agg_data['y'].fillna(0)

In [7]:
natal_events = pd.DataFrame({
    'holiday': ['Christmas'] * 6,
    'ds': pd.to_datetime(['2011-12-25', '2012-12-25', '2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25']),
    'lower_window': [-2] * 6,
    'upper_window': [1] * 6
})

ramadan_events = pd.DataFrame({
    'holiday': ['Ramadan'] * 6,
    'ds': pd.to_datetime(['2011-08-01', '2012-07-20', '2013-07-09', '2014-06-28', '2015-06-18', '2016-06-06']),
    'lower_window': [0] * 6,
    'upper_window': [30] * 6
})

events_df = pd.concat([natal_events, ramadan_events])
events_df = events_df[['ds', 'holiday', 'lower_window', 'upper_window']].drop_duplicates()

In [ ]:
model = Prophet(holidays=events_df, yearly_seasonality=True, weekly_seasonality=True)
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
model.fit(agg_data)

In [9]:
future = model.make_future_dataframe(periods=28, freq='D')  # Previsão diária para 28 dias
forecast = model.predict(future)

In [ ]:
fig = model.plot(forecast)
plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], color='gray', alpha=0.2)
plt.show()

In [11]:
forecast.to_csv('previsoes_vendas_28_dias.csv', index=False)

In [ ]:
fig2 = model.plot_components(forecast)